**Problem 3, IMO 1988**

A function f is defined on the positive integers by: f(1) = 1; f(3) = 3; f(2n) = f(n), f(4n + 1) = 2f(2n + 1) - f(n), and f(4n + 3) = 3f(2n + 1) - 2f(n) for all positive integers n. Determine the number of positive integers n less than or equal to 1988 for which f(n) = n. (from [here](https://prase.cz/kalva/imo/isoln/isoln883.html))

In [1]:
function f(n::Int)
    if n == 1 
        1
    elseif n == 3
        3
    elseif mod(n, 2) == 0
        f(n÷2)
    elseif mod(n, 4) == 1
        m = (n-1)÷4
        2*f(2*m+1) - f(m)
    elseif mod(n, 4) == 3
        m = (n-3)÷4
        3*f(2*m+1) - 2*f(m)
    end
end

f (generic function with 1 method)

In [2]:
isfixedpoint(f,x) = (f(x) == x) ? true : false

isfixedpoint (generic function with 1 method)

***Calculating the IMO problem solution***

In [10]:
mapreduce(x -> isfixedpoint(f,x), +, 1:1988)

92

***Playing with performance***

In [11]:
@time mapreduce(x -> isfixedpoint(f,x), +, 1:1001988)

 19.096489 seconds (73.44 k allocations: 3.515 MiB)


2001

In [14]:
@time mapreduce(x -> (f(x) == x), +, 1:1001988)

 19.251413 seconds (70.55 k allocations: 3.400 MiB)


2001

In [21]:
res = 0
@time for i in 1:1001988
    if f(i) == i
        global res
        res = res +1
    end
end
println(res)

 19.110554 seconds (1.49 k allocations: 23.281 KiB)
2001


**Distributed version**
Using all CPU cores on the local system, can be run on clusters!
Note that global definitions aren't automatically synchronised between workers, thus @everywhere

In [17]:
rmprocs(workers())
addprocs() # Adds number of local cores by default
println("Number of processors: ", nprocs())

@everywhere function f(n::Int)
    if n == 1 
        1
    elseif n == 3
        3
    elseif mod(n, 2) == 0
        f(n÷2)
    elseif mod(n, 4) == 1
        m = (n-1)÷4
        2*f(2*m+1) - f(m)
    elseif mod(n, 4) == 3
        m = (n-3)÷4
        3*f(2*m+1) - 2*f(m)
    end
end

@time @distributed (+) for x = 1:1001988
    x == f(x) ? 1 : 0
end


Number of processors: 9
  6.032024 seconds (87.71 k allocations: 4.323 MiB)


2001